# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [16]:
def exp_return(data, P):
    # Write your code here
    # exp_return(data, 40) should output 0.07500000000000002
    data['Price in 1 Year'] = pd.to_numeric(data['Price in 1 Year'])
    data['Probability'] = pd.to_numeric(data['Probability'])
    
    data['Return'] = (data['Price in 1 Year'] - P) / P
    expected_return = sum(data['Return'] * data['Probability'])
    return expected_return

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [40]:
def volatility(data, P):
    # Write your code here
    # volatility(data,40) should output 0.21301408404140795
    
    expected_return = exp_return(data, P)
    
    variance = sum(data['Probability'] * ((data['Price in 1 Year'] - P) / P - expected_return) ** 2)
    volatility = variance ** 0.5
    
    return volatility

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [42]:
def interval(data, P):
    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    expected_return = exp_return(data, P)
    vol = volatility(data, P)
    #interval = 0.95
    Z = 1.96
    
    std = vol / np.sqrt(len(data))
    
    lower_bound = expected_return - (Z * std)
    upper_bound = expected_return + (Z * std)
    return [lower_bound, upper_bound]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [44]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    return rf + (beta * (market_return - rf))

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [48]:
def clean_data(path):
    # Write your code here
    # This should yield data with 403316 rows × 4 columns
    df = pd.read_csv(path)
    df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
    df.dropna(subset=['RET'], inplace=True)
    df = df.groupby('CUSIP').filter(lambda x: len(x) >= 500)
    return df


Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [57]:
def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    
    output = pd.DataFrame(data = {
        "CUSIP" : [],
        "Average Return" : []
    })
    
    avg_ret = data.groupby('CUSIP')['RET'].mean()
    output = pd.DataFrame({'CUSIP': avg_ret.index, 'Average Return': avg_ret.values})
    
    return output

In [61]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    
    output = pd.DataFrame(data = {
        "CUSIP" : [],
        "Volatility" : []
    })
    
    vol = data.groupby('CUSIP')['RET'].std()
    output = pd.DataFrame({'CUSIP': vol.index, 'Volatility': vol.values})
    
    
    return output

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [67]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    merge = pd.merge(ret_data, vol_data, on='CUSIP')
    corr_val = merge['Average Return'].corr(merge['Volatility'])
    
    return corr_val